'''Author- Prakash Chandra Chhipa, Email- prakash.chandra.chhipa@ltu.se/prakash.chandra.chhipa@gmail.com, Year- 2022'''

In [ ]:
from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50
import torch

In [2]:
import os, sys
import numpy as np
import PIL
from PIL import Image
sys.path.append(os.path.dirname("/home/BC_SSL/src/supervised"))
from supervised.core.models import ResNetDilated_BACH
sys.path.append(os.path.dirname("/home/BC_SSL/src/supervised/apply"))
from supervised.apply.transform import resize_transform_bach_224
#os.environ["CUDA_VISIBLE_DEVICES"] = "7"

In [3]:
def force_cudnn_initialization(device):
    s = 32
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=device), torch.zeros(s, s, s, s, device=device))

In [4]:
device = torch.device('cuda:0')
force_cudnn_initialization(device=device)

In [5]:
downstream_task_model = ResNetDilated_BACH(version=int(50), num_classes=4)
#model = resnet50(pretrained=True) 
target_layers = [downstream_task_model.model.layer4[-1]]

In [6]:
#downstream_task_model = downstream_task_model.to(device)

In [7]:
#give model path
downstream_task_model.load_state_dict(torch.load("", map_location=device))


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 31.75 GiB total capacity; 86.63 MiB already allocated; 20.75 MiB free; 92.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
targets = [ClassifierOutputTarget(4)]


In [ ]:
input_rgb = PIL.Image.open("") #give randomly chosen input image full path
input_tensor = resize_transform_bach_224(np.array(input_rgb))
input_tensor = input_tensor.unsqueeze(0)

In [ ]:
input_tensor = input_tensor.to(device)

In [ ]:
cam = GradCAM(model=downstream_task_model.model, target_layers=target_layers, use_cuda=True)
#cam = cam.to(device)
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(input_rgb, grayscale_cam, use_rgb=True)
visualization.save("/home/datasets/BACH/cam.png")

RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED